# Questioning Barbie and Oppenheimer Through the Use of Agents

In the following notebook we will build an application that queries both the Barbie and Oppenheimer movies Wikipedia pages, as well as their reviews.

The main focus of this notebook is to showcase a brief introduction to Agents.

## Build 🏗️

There are 3 main tasks in this notebook:

1. Contruct a Barbie retriever
2. Construct an Oppenheimer retriever
3. Combine the two and allow users to query both resources from a single input through the use of Agents

## Ship 🚢

Based on Tuesday's session - construct a Chainlit (or Gradio) application that allows users to interface with the application.

## Share 🚀

Make a social media post about your final application.

### Dependencies

As always, let's start with some dependencies!

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/LLMOps-AIMakerpace/LLM-Ops-Cohort-1-main/Week 1/Thursday

/content/drive/MyDrive/LLMOps-AIMakerpace/LLM-Ops-Cohort-1-main/Week 1/Thursday


In [3]:
!pip install -q -U langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00


In [4]:
import getpass
import os
openai_api_key = getpass.getpass("Enter your OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

Enter your OpenAI API Key: ··········


### LLM

We will be leveraging OpenAI's `gpt-3.5-turbo` throughout the notebook, and we can keep it consistent throughout!

In [5]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature = 0)

### Data Collection and Transformation

We'll be leveraging the `WikipediaLoader` tool to collect information from Wikipedia.

Be sure to set the `doc_content_chars_max` parameter so that you capture the *entire* Wikipedia article content.

In [6]:
!pip install -q -U wikipedia

  Preparing metadata (setup.py) ... done


In [7]:
from langchain.document_loaders import WikipediaLoader, CSVLoader

barbie_wikipedia_docs = WikipediaLoader(
    query="Barbie (film)",
    load_max_docs=1000,
    doc_content_chars_max=100
    ).load()

barbie_csv_docs = CSVLoader(
    file_path="/content/drive/MyDrive/LLMOps-AIMakerpace/LLM-Ops-Cohort-1-main/Week 1/Thursday/barbie_data/barbie.csv",
    source_column="Review_Url"
    ).load()

Since we'll be using same format source documentation separated by topic, we can save ourselves some extra effort and set up our splitters once.

We're going to leverage the `RecursiveCharacterTextSplitter` again, this time paying close attention to the format our Wikipedia articles and reviews are in so we can be sure to chunk them appropritately.

> HINT: You can pass a list of separators when you intialize your `RecursiveTextSplitter`! They are acted on in order of element 0 -> element len(list).

RELEVANT DOCS:
- [`RecursiveCharacterTextSplitter`](https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html#langchain.text_splitter.RecursiveCharacterTextSplitter)

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

wikipedia_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100,
    length_function = len,
    is_separator_regex= False,
    separators = ["\n\n", "\n", ". "] # keep headings, then paragraphs, then sentences
)

csv_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100,
    length_function = len,
    is_separator_regex= False,
    separators = ["\n", ". "]  # keep paragraphs, then sentences
)

chunked_barbie_wikipedia_docs = wikipedia_text_splitter.transform_documents(barbie_wikipedia_docs) ### YOUR CODE HERE
chunked_barbie_csv_docs = csv_text_splitter.transform_documents(barbie_csv_docs) ### YOUR CODE HERE

#### Retrieval and Embedding Strategy

We've already discussed the useful application of `CacheBackedEmbeddings`, so let's do it again!

RELEVANT DOCS:
- [`CacheBackedEmbeddings`](https://api.python.langchain.com/en/latest/embeddings/langchain.embeddings.cache.CacheBackedEmbeddings.html#langchain-embeddings-cache-cachebackedembeddings)

In [9]:
!pip install -q -U rank_bm25 tiktoken faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 38.1 MB/s eta 0:00:00


In [10]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

# set up cached embeddings store
store = LocalFileStore("./cache/")

core_embeddings_model = OpenAIEmbeddings()

embedder = CacheBackedEmbeddings.from_bytes_store(core_embeddings_model, store,
    namespace=core_embeddings_model.model
)

We'll implement a `FAISS` vectorstore, and create a retriever from it.

In [11]:
barbie_csv_faiss_retriever = FAISS.from_documents(chunked_barbie_wikipedia_docs, embedder)


There are a number of excellent options to retrieve documents - we'll be looking at an additional example today, which is called the `EnsembleRetriever`.

The method this is using is outlined in [this paper](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf).

The brief explanation is:

1. We collect results from two different retrieval methods over the same corpus
2. We apply a reranking algorithm to rerank our source documents to be the *most relevant* without losing specific or potentially low-ranked information rich documents
3. We feed the top-k results into the LLM with our query as context.

> HINT: Your weight list should be of type `List[float]` and the `sum(List[float])` should be `1`.

In [12]:
# set up BM25 retriever
barbie_wikipedia_bm25_retriever = BM25Retriever.from_documents(
    documents=chunked_barbie_wikipedia_docs
)
barbie_wikipedia_bm25_retriever.k = 1

# set up FAISS vector store
barbie_wikipedia_faiss_store = FAISS.from_documents(documents=chunked_barbie_wikipedia_docs, embedding=core_embeddings_model)

barbie_wikipedia_faiss_retriever = barbie_wikipedia_faiss_store.as_retriever(search_kwargs={"k": 1})

# set up ensemble retriever
barbie_ensemble_retriever = EnsembleRetriever(
    retrievers=[barbie_wikipedia_bm25_retriever, barbie_wikipedia_faiss_retriever],
    weights=[0.5, 0.5]
)

#### Retrieval Agent

We can create a simple conversational retrieval Agent by using the built-ins provided by LangChain!

> HINT: Be sure to provide good natural language descriptions of what the tool should be used for to get the best results.

RELEVANT DOCS:
- [`create_retriever_tool`](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool.html#langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool)

In [13]:
from langchain.agents.agent_toolkits import create_retriever_tool

barbie_wikipedia_retrieval_tool = create_retriever_tool(
    retriever=barbie_wikipedia_bm25_retriever,
    name="Barbie_Wikipedia_Retrieval",
    description="A tool for retrieving information from Barbie's Wikipedia."
)

barbie_csv_retrieval_tool = create_retriever_tool(
    retriever=barbie_csv_faiss_retriever.as_retriever(search_kwargs={"k": 1}), # converting barbie_csv_faiss_retriever from vector store to retriever
    name="Barbie_CSV_Retrieval",
    description="A tool for retrieving information from Barbie's CSV documents."
)

barbie_retriever_tools = [barbie_wikipedia_retrieval_tool, barbie_csv_retrieval_tool]

Now that we've created our tools, we can combined them into an agent!

RELEVANT DOCS:
- [`create_conversational_retrieval_agent`](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.openai_functions.create_conversational_retrieval_agent.html#langchain.agents.agent_toolkits.conversational_retrieval.openai_functions.create_conversational_retrieval_agent)

In [14]:
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

barbie_retriever_agent_executor = create_conversational_retrieval_agent(
    llm=llm,
    tools=barbie_retriever_tools,
    verbose=True
)

In [15]:
barbie_retriever_agent_executor({"input" : "Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?"})



> Entering new AgentExecutor chain...

Invoking: `Barbie_Wikipedia_Retrieval` with `Barbie`


[Document(page_content='Barbie: The Album is the soundtrack album for the 2023 film Barbie, directed by Greta Gerwig. It was', metadata={'title': 'Barbie: The Album', 'summary': 'Barbie: The Album is the soundtrack album for the 2023 film Barbie, directed by Greta Gerwig. It was released by Atlantic Records on July 21, 2023, the same day as the film\'s North American theatrical release. The album was promoted by five singles: "Dance the Night", "Barbie World", "Speed Drive", "What Was I Made For?", and "Choose Your Fighter". The album debuted at number one in Australia, Canada, the Netherlands, and New Zealand.\n\n', 'source': 'https://en.wikipedia.org/wiki/Barbie:_The_Album'})]I apologize, but I couldn't find specific information about the philosophical nature of the Barbie movie. The information I found is about the Barbie: The Album soundtrack for the 2023 film directed by Greta Gerwig. I

{'input': 'Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?',
 'chat_history': [HumanMessage(content='Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?', additional_kwargs={}, example=False),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'Barbie_Wikipedia_Retrieval', 'arguments': '{\n  "__arg1": "Barbie"\n}'}}, example=False),
  FunctionMessage(content='[Document(page_content=\'Barbie: The Album is the soundtrack album for the 2023 film Barbie, directed by Greta Gerwig. It was\', metadata={\'title\': \'Barbie: The Album\', \'summary\': \'Barbie: The Album is the soundtrack album for the 2023 film Barbie, directed by Greta Gerwig. It was released by Atlantic Records on July 21, 2023, the same day as the film\\\'s North American theatrical release. The album was promoted by five singles: "Dance the Night", "Barbie 

In [16]:
barbie_retriever_agent_executor({"input" : "What is a very quick summary of the plot of the Barbie movie?"})



> Entering new AgentExecutor chain...

Invoking: `Barbie_Wikipedia_Retrieval` with `Barbie (film)`


[Document(page_content='Barbie: The Album is the soundtrack album for the 2023 film Barbie, directed by Greta Gerwig. It was', metadata={'title': 'Barbie: The Album', 'summary': 'Barbie: The Album is the soundtrack album for the 2023 film Barbie, directed by Greta Gerwig. It was released by Atlantic Records on July 21, 2023, the same day as the film\'s North American theatrical release. The album was promoted by five singles: "Dance the Night", "Barbie World", "Speed Drive", "What Was I Made For?", and "Choose Your Fighter". The album debuted at number one in Australia, Canada, the Netherlands, and New Zealand.\n\n', 'source': 'https://en.wikipedia.org/wiki/Barbie:_The_Album'})]
Invoking: `Barbie_Wikipedia_Retrieval` with `Barbie (film)`


[Document(page_content='Barbie: The Album is the soundtrack album for the 2023 film Barbie, directed by Greta Gerwig. It was', metadata={'title': '

{'input': 'What is a very quick summary of the plot of the Barbie movie?',
 'chat_history': [HumanMessage(content='Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?', additional_kwargs={}, example=False),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'Barbie_Wikipedia_Retrieval', 'arguments': '{\n  "__arg1": "Barbie"\n}'}}, example=False),
  FunctionMessage(content='[Document(page_content=\'Barbie: The Album is the soundtrack album for the 2023 film Barbie, directed by Greta Gerwig. It was\', metadata={\'title\': \'Barbie: The Album\', \'summary\': \'Barbie: The Album is the soundtrack album for the 2023 film Barbie, directed by Greta Gerwig. It was released by Atlantic Records on July 21, 2023, the same day as the film\\\'s North American theatrical release. The album was promoted by five singles: "Dance the Night", "Barbie World", "Speed Drive", "What Was I Made For?", and "Choose Your 

### Oppenheimer Retrieval System

We're going to repourpose some of what we created previously, but this time we'll explore a different multi-source retrieval system.

In [17]:
oppenheimer_wikipedia_docs = WikipediaLoader(
    query="J. Robert Oppenheimer",
    load_max_docs=1000,
    doc_content_chars_max=100
).load()

oppenheimer_csv_docs = CSVLoader(
    file_path="/content/drive/MyDrive/LLMOps-AIMakerpace/LLM-Ops-Cohort-1-main/Week 1/Thursday/oppenheimer_data/oppenheimer.csv", # Modify the file path to the correct location
    source_column="Review_Url"
).load()

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [18]:
chunked_opp_wikipedia_docs = wikipedia_text_splitter.transform_documents(oppenheimer_wikipedia_docs)
chunked_opp_csv_docs = csv_text_splitter.transform_documents(oppenheimer_csv_docs)

In [19]:
opp_csv_faiss_retriever = FAISS.from_documents(chunked_opp_wikipedia_docs, core_embeddings_model)

# set up BM25 retriever
opp_wikipedia_bm25_retriever = BM25Retriever.from_documents(documents=chunked_opp_wikipedia_docs)
opp_wikipedia_bm25_retriever.k = 1

# set up FAISS vector store
opp_wikipedia_faiss_store = FAISS.from_documents(documents=chunked_opp_wikipedia_docs, embedding=core_embeddings_model)
opp_wikipedia_faiss_retriever = opp_wikipedia_faiss_store.as_retriever(search_kwargs={"k": 1})

# set up ensemble retriever
opp_ensemble_retriever = EnsembleRetriever(
    retrievers=[opp_wikipedia_bm25_retriever, opp_wikipedia_faiss_retriever],
    weights=[0.5, 0.5]
)

#### Multi-source chain

We're going to allow the LLM to decide which information is most -> least valuable.

The way we'll do this is with LangChain's rather powerful "Expression Language"!

> HINT: You can leverage [this](https://python.langchain.com/docs/use_cases/question_answering/how_to/multiple_retrieval) resource if you get stuck - but experiment with different prompts/formats.

In [20]:
from langchain.prompts import ChatPromptTemplate

system_message = """Use the information from the below two sources to answer any questions.

Source 1: public user reviews about the Oppenheimer movie
<source1>
{source1}
</source1>

Source 2: the wikipedia page for the Oppenheimer movie including the plot summary, cast, and production information
<source2>
{source2}
</source2>
"""

prompt = ChatPromptTemplate.from_messages([("system", system_message), ("human", "{question}")])

In [21]:
oppenheimer_multisource_chain = {
    "source1": (lambda x: x["question"]) | opp_ensemble_retriever,
    "source2": (lambda x: x["question"]) | opp_ensemble_retriever, # changed opp_csv_faiss_retriever to opp_ensemble_retriever
    "question": lambda x: x["question"],
} | prompt | llm

In [22]:
oppenheimer_multisource_chain.invoke({"question" : "What did people think of the Oppenheimer movie?"})

AIMessage(content='There is no specific information available about public opinions on the Oppenheimer movie. The provided sources do not contain any user reviews or feedback about the movie.', additional_kwargs={}, example=False)

# Agent Creation

Now we can finally start building our Agent!

The first thing we'll need to do is provide our Agent a Toolbelt. (list of tools). Much like Batman, our LLM-powered Agent can use these tools as it sees fit.

While the examples we're constructing in this notebook are straightforward for brevity and simplicities sake - there is no limit to what you can build with Agents, as we'll see as we progress through the program.

So, let's begin by setting up our Tools!

You'll notice that we have to set up a function to allow our `OppenheimerInfo` tool to interface with the Agent - this is due to it have a specific required input. Creating custom tools is a pattern that you'll want to grow acustomed to as you use LangChain more and more.

In [31]:
from langchain.agents import Tool

def query_oppenheimer(input):
    return oppenheimer_multisource_chain.invoke({"question" : input})

tools = [
    Tool(
        name = "BarbieInfo",
        func=barbie_retriever_agent_executor.invoke,
        description="useful for when you need to answer questions about Barbie. Input should be a fully formed question."
    ),
    Tool(
        name = "OppenheimerQASystem",
        func= query_oppenheimer,
        description="useful for when you need to answer questions about Oppenheimer movie. Input should be a fully formed question."
    ),
]

Now that we've set up our Agents toolbelt, let's set up the LLM that will be powering it!

I would suggest playing around with these prompts - and experiments to find what works best for you.

RELEVANT DOCS:
- [`ZeroShotAgent`](https://api.python.langchain.com/en/latest/agents/langchain.agents.mrkl.base.ZeroShotAgent.html#langchain-agents-mrkl-base-zeroshotagent)

In [32]:
from langchain.agents import ZeroShotAgent, AgentExecutor

prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    prefix=prefix,
    suffix=suffix,
    tools=tools
)

In [33]:
from langchain import LLMChain

llm_chain = LLMChain(llm=llm, prompt=prompt)

All that's left to do now is create our `ZeroShotAgent` and our `AgentExecutor`, which are the "reasoner" and "actor" halfs of the `ReAct` method of Agent implementation.

Read all about the `ReAct` framework [here](https://react-lm.github.io/)

In [34]:
barbenheimer_agent = ZeroShotAgent(
    llm_chain=llm_chain,
    tools=tools,
    verbose=True)

barbenheimer_agent_chain = AgentExecutor.from_agent_and_tools(
    agent=barbenheimer_agent,
    tools=tools,
    verbose=True)

## Conclusion

All that is left to do now, is feed inputs to your Agent and watch it work!

Remember to use the `{"input" : "YOUR QUERY HERE"}` format when prompting the Agent.

In [35]:
barbenheimer_agent_chain.invoke({"input" : "What did people like about the Barbie movie?"})



> Entering new AgentExecutor chain...
Thought: I need to find out what people liked about the Barbie movie.
Action: BarbieInfo
Action Input: What did people like about the Barbie movie?

> Entering new AgentExecutor chain...

Invoking: `Barbie_Wikipedia_Retrieval` with `Barbie (film)`


[Document(page_content='Barbie: The Album is the soundtrack album for the 2023 film Barbie, directed by Greta Gerwig. It was', metadata={'title': 'Barbie: The Album', 'summary': 'Barbie: The Album is the soundtrack album for the 2023 film Barbie, directed by Greta Gerwig. It was released by Atlantic Records on July 21, 2023, the same day as the film\'s North American theatrical release. The album was promoted by five singles: "Dance the Night", "Barbie World", "Speed Drive", "What Was I Made For?", and "Choose Your Fighter". The album debuted at number one in Australia, Canada, the Netherlands, and New Zealand.\n\n', 'source': 'https://en.wikipedia.org/wiki/Barbie:_The_Album'})]
Invoking: `Barbie_Wiki

{'input': 'What did people like about the Barbie movie?',
 'output': "I couldn't find specific information about what people liked about the Barbie movie."}

In [36]:
barbenheimer_agent_chain.run({"input" : "What did people like about the Oppenheimer movie?"})



> Entering new AgentExecutor chain...
Thought: I don't have enough information about the Oppenheimer movie to answer this question.
Action: OppenheimerQASystem
Action Input: What did people like about the Oppenheimer movie?
Observation: content='There is no specific information available about the opinions of people regarding the Oppenheimer movie in the provided sources.' additional_kwargs={} example=False
Thought:I don't have enough information about the Oppenheimer movie to answer this question.
Action: BarbieInfo
Action Input: What did people like about the Barbie movies?

> Entering new AgentExecutor chain...

Invoking: `Barbie_Wikipedia_Retrieval` with `Barbie (film series)`


[Document(page_content='Barbie: The Album is the soundtrack album for the 2023 film Barbie, directed by Greta Gerwig. It was', metadata={'title': 'Barbie: The Album', 'summary': 'Barbie: The Album is the soundtrack album for the 2023 film Barbie, directed by Greta Gerwig. It was released by Atlantic Recor

"The Barbie film series has been popular among audiences for various reasons. Some of the aspects that people have liked about the Barbie movies include positive messages about friendship, self-acceptance, and following one's dreams, empowering characters, engaging storylines, catchy songs, and visual appeal. These movies often promote values such as kindness, courage, and perseverance, feature strong and independent female characters, offer imaginative storylines with elements of adventure, fantasy, and romance, have memorable songs, and are visually appealing with vibrant animation and colorful settings. It's important to note that the specific reasons people like the Barbie movies may vary from person to person, but these are some common aspects that have contributed to their popularity."

In [37]:
barbenheimer_agent_chain.run({"input" : "Did the movies Barbie and Oppenheimer share similar themes or ideas?"})



> Entering new AgentExecutor chain...
Thought: I need to find information about both movies to determine if they share similar themes or ideas.
Action: BarbieInfo
Action Input: "What are the themes in the movie Barbie?"

> Entering new AgentExecutor chain...

Invoking: `Barbie_Wikipedia_Retrieval` with `Barbie (film)`


[Document(page_content='Barbie: The Album is the soundtrack album for the 2023 film Barbie, directed by Greta Gerwig. It was', metadata={'title': 'Barbie: The Album', 'summary': 'Barbie: The Album is the soundtrack album for the 2023 film Barbie, directed by Greta Gerwig. It was released by Atlantic Records on July 21, 2023, the same day as the film\'s North American theatrical release. The album was promoted by five singles: "Dance the Night", "Barbie World", "Speed Drive", "What Was I Made For?", and "Choose Your Fighter". The album debuted at number one in Australia, Canada, the Netherlands, and New Zealand.\n\n', 'source': 'https://en.wikipedia.org/wiki/Barbie:_Th

"I couldn't find specific information about the themes in the Barbie movie, but the Barbie movies are known for their positive messages, empowering characters, engaging storylines, catchy songs, and visual appeal."

## Next Steps

It's time to build a Chainlit (or Gradio) application and host it on Hugging Face Spaces! :ship: